# CSC6711 Project 3 - Non-Personalized Recommendations

* **Author**: Jacob Buysse

This notebook is an analysis of the non-personalized predictions from the 4 datasets from Project 2 and how good they are at predicing the individual users actual ratings.  The files are located in the `datasets` subdirectory:
* MovieLens - `movielens_25m.feather` (Movies)
* Netflix Prize - `netflix_prize.feather` (Movies and TV Shows)
* Yahoo! Music R2 - `yahoo_r2_songs.subsampled.feather` (Songs)
* BoardGameGeek - `boardgamegeek.feather` (Board Games)

We will be using the following libraries:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

Let us configure matplotlib for readable labels, high resolution, and automatic layout.

In [2]:
matplotlib.rc('axes', labelsize=16)
matplotlib.rc('figure', dpi=150, autolayout=True)

## Datasets

Let us load the databases using Pandas.  We know from Project 2 that the contents are structured identically:

* `df1` - MovieLens
* `df2` - Netflix
* `df3` - Yahoo Music
* `df4` - BoardGameGeek

In each file, we have `item_id`, `user_id`, and `rating`.

In [3]:
df1 = pd.read_feather('./datasets/movielens_25m.feather')
df2 = pd.read_feather('./datasets/netflix_prize.feather')
df3 = pd.read_feather('./datasets/yahoo_r2_songs.subsampled.feather')
df4 = pd.read_feather('./datasets/boardgamegeek.feather')

In [4]:
df1.info()
print("")
df2.info()
print("")
df3.info()
print("")
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24890583 entries, 0 to 25000094
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   item_id  int64  
 1   user_id  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 759.6 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51031355 entries, 0 to 51031354
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   item_id  int64
 1   user_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 1.1 GB

<class 'pandas.core.frame.DataFrame'>
Index: 6937275 entries, 35 to 1999917
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   item_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 211.7 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   item_id  int64  
 1   user_id  object 
 2   rating   float64
dt